# Obtención de datos utilizando la API de Spotify

La API de Spotify es una herramienta poderosa que nos acceder a una amplia gama de datos relacionados con la música disponible en la plataforma de Spotify. A través de esta API, es posible obtener información detallada sobre canciones, álbumes, artistas, listas de reproducción...

In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import os

In [2]:
# Ocultamos nuestras credenciales y las definimos
load_dotenv()

client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

In [3]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [4]:
def buscar_listas_reproduccion(query, limit=10):
    # Buscar listas de reproducción con el término dado
    resultados = sp.search(q=query, type='playlist', limit=limit)
    playlists = resultados['playlists']['items']
    
    # Crear una lista para almacenar información de las playlists
    playlist_data = []
    
    for playlist in playlists:
        playlist_info = {
            'nombre_playlist': playlist['name'],
            'id_playlist': playlist['id'],
            'total_tracks': playlist['tracks']['total']
        }
        playlist_data.append(playlist_info)
    
    return playlist_data

# Buscar listas con temas de anuncios de televisión
query = "anuncios TV"
playlists = buscar_listas_reproduccion(query, limit=10)


In [5]:
def obtener_detalles_de_tracks(playlists):
    tracks_data = []
    
    for playlist in playlists:
        tracks = sp.playlist_tracks(playlist['id_playlist'])['items']
        for item in tracks:
            track = item['track']
            
            # Verificar si el track es None
            if track is None:
                continue
            
            # Obtener características adicionales de audio, si están disponibles
            audio_features = sp.audio_features(track['id'])[0] if track['id'] else None
            
            # Verificar que audio_features no sea None antes de acceder a sus valores
            if audio_features is None:
                audio_features = {}
            
            track_info = {
                'nombre_cancion': track.get('name', ''),  # Utiliza una cadena vacía si el nombre es None
                'duracion_ms': track.get('duration_ms', 0),  # Utiliza 0 si no se encuentra la duración
                'popularidad': track.get('popularity', 0),  # Utiliza 0 si no hay popularidad
                'acousticness': audio_features.get('acousticness', 0.0),  # Utiliza un valor por defecto si no hay audio features
                'danceability': audio_features.get('danceability', 0.0),
                'energy': audio_features.get('energy', 0.0),
                'instrumentalness': audio_features.get('instrumentalness', 0.0),
                'liveness': audio_features.get('liveness', 0.0),
                'loudness': audio_features.get('loudness', 0.0),
                'speechiness': audio_features.get('speechiness', 0.0),
                'tempo': audio_features.get('tempo', 0.0),
                'valence': audio_features.get('valence', 0.0),
                'key': audio_features.get('key', 0),
                'mode': audio_features.get('mode', 0),
                'time_signature': audio_features.get('time_signature', 4)
            }
            tracks_data.append(track_info)
    
    return pd.DataFrame(tracks_data)

# Crear el DataFrame con todos los detalles de los tracks
df_tracks_detalles = obtener_detalles_de_tracks(playlists)

# Mostrar las primeras filas del DataFrame
df_tracks_detalles


,nombre_cancion,duracion_ms,popularidad,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,time_signature
0,I Just Want To Make Love To You - Single Version,185626,63,0.772,0.642,0.523,0.000003,0.5270,-8.579,0.1100,101.635,0.654,3,1,4
1,Build Me Up Buttercup - Mono,180133,79,0.196,0.640,0.664,0.000000,0.3100,-4.730,0.0289,134.002,0.879,0,1,4
2,Boombastic,249933,70,0.242,0.867,0.538,0.000017,0.3160,-16.183,0.3610,158.328,0.781,2,1,4
3,Aquarius/Let The Sunshine In (The Flesh Failur...,289293,57,0.445,0.293,0.740,0.001520,0.0998,-6.098,0.0399,118.225,0.514,7,1,4
4,Sticks and Stones - 2007 Remaster,143040,0,0.345,0.577,0.848,0.000011,0.0350,-5.765,0.2340,87.584,0.922,2,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,(Theme From) The Monkees - TV Version; 2006 Re...,50600,14,0.119,0.459,0.429,0.000507,0.5800,-9.816,0.0301,81.577,0.722,9,0,4
707,Rawhide,53506,11,0.885,0.179,0.538,0.750000,0.0441,-13.402,0.0552,205.228,0.663,5,0,4
708,The Facts of Life,53880,0,0.657,0.525,0.596,0.037900,0.0463,-4.185,0.0403,139.850,0.921,5,1,4
709,Night Court,183106,0,0.347,0.741,0.450,0.647000,0.0704,-13.478,0.0303,98.244,0.875,5,0,4
